In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import STL10
from torch.utils.data import DataLoader
from torchmetrics import Accuracy, F1Score
import torch.optim as optim
from tqdm.notebook import tqdm
import random
import seaborn as sns
from PIL import ImageFilter, ImageOps, Image
import torch.optim as optim
from tqdm import tqdm
from matplotlib import pyplot as plt
from torchvision.models import resnet18
import torch.nn as nn
import torch.utils.data as data_utils
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
def get_resnet_backbone():
    model = resnet18(pretrained=False)
    in_features = model.fc.in_features
    model.fc = torch.nn.Identity()
    return model, in_features

In [3]:
class ResNet18Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(4608, 64)

    def _make_layer(self, out_channels, blocks, stride):
        downsample = None
        if (
            stride != 1
            or self.in_channels != out_channels * BasicEncoderBlock.expansion
        ):
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    out_channels * BasicEncoderBlock.expansion,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels * BasicEncoderBlock.expansion),
            )

        layers = []
        layers.append(
            BasicEncoderBlock(self.in_channels, out_channels, stride, downsample)
        )
        self.in_channels = out_channels * BasicEncoderBlock.expansion
        for _ in range(1, blocks):
            layers.append(BasicEncoderBlock(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        (
            n,
            _,
            _,
            _,
        ) = x.shape
        z = x.reshape(n, -1)
        z = self.linear(z)
        return z


class BasicEncoderBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            out_channels, out_channels, kernel_size=3, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


In [4]:
class GaussianBlur(object):
    def __init__(self, p=0.5, radius_min=0.1, radius_max=2.):
        self.prob = p
        self.radius_min = radius_min
        self.radius_max = radius_max

    def __call__(self, img):
        do_it = random.random() <= self.prob
        if not do_it:
            return img

        return img.filter(
            ImageFilter.GaussianBlur(
                radius=random.uniform(self.radius_min, self.radius_max)
            )
        )
        
class Solarization(object):
    def __init__(self, p):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            return ImageOps.solarize(img)
        else:
            return img

In [5]:
class DataAugmentationDINO(object):
    def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
        flip_and_color_jitter = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
        ])
        normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

        # first global crop
        self.global_transfo1 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            GaussianBlur(1.0),
            normalize,
        ])
        # second global crop
        self.global_transfo2 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            GaussianBlur(0.1),
            Solarization(0.2),
            normalize,
        ])
        # transformation for the local small crops
        self.local_crops_number = local_crops_number
        self.local_transfo = transforms.Compose([
            transforms.RandomResizedCrop(96, scale=local_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            GaussianBlur(p=0.5),
            normalize,
        ])

    def __call__(self, image):
        crops = []
        crops.append(self.global_transfo1(image))
        crops.append(self.global_transfo2(image))
        for _ in range(self.local_crops_number):
            crops.append(self.local_transfo(image))
        return crops

In [6]:
transform = DataAugmentationDINO(
        (0.4, 1.), # global crops scale
        (0.05, 0.4), # local crops number
        8 # local crops scale
    )

In [7]:
class ReconstructionDataModule(pl.LightningDataModule):
    def __init__(self, path, preprocess, batch_size=128):
        super().__init__()
        self.path = path
        self.batch_size = batch_size
        self.preprocess = preprocess

    def setup(self, stage=None):
        generator = torch.Generator().manual_seed(42)
        self.dataset = STL10(self.path, split="unlabeled", transform=self.preprocess)
        self.train_dataset, self.val_dataset = data_utils.random_split(
            self.dataset, [0.8, 0.2], generator=generator
        )
        self.train_dataset = data_utils.Subset(self.train_dataset, range(0, 10000))
        self.val_dataset = data_utils.Subset(self.val_dataset, range(0, 1000))

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

In [21]:
class DINOHead(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_dim=64, use_bn=False):
        super(DINOHead, self).__init__()
        layers = [
            nn.Linear(in_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, out_dim)
        ]
        if use_bn:
            layers.insert(1, nn.BatchNorm1d(hidden_dim))
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        return self.mlp(x)

class DINOLoss(nn.Module):
    def __init__(self, out_dim, teacher_temp=0.04, student_temp=0.1, center_momentum=0.996):
        super(DINOLoss, self).__init__()
        self.register_buffer("center", torch.zeros(1, out_dim))
        self.teacher_temp = teacher_temp
        self.student_temp = student_temp
        self.center_momentum = center_momentum

    def forward(self, student_output, teacher_output):
        teacher_output = F.softmax((teacher_output - self.center) / self.teacher_temp, dim=-1).detach()
        student_output = F.log_softmax(student_output / self.student_temp, dim=-1)
        loss = torch.mean(torch.sum(-teacher_output * student_output, dim=-1))
        self.update_center(teacher_output)
        return loss

    @torch.no_grad()
    def update_center(self, teacher_output):
        batch_center = torch.mean(teacher_output, dim=0, keepdim=True)
        self.center = self.center * self.center_momentum + batch_center * (1 - self.center_momentum)

class DINO(nn.Module):
    def __init__(self, backbone, head):
        super(DINO, self).__init__()
        self.backbone = backbone
        self.head = head

    def forward(self, x):
        features = self.backbone(x)
        return self.head(features)

# encoder = ResNet18Encoder()
encoder, in_features = get_resnet_backbone()
student_head = DINOHead(in_dim=in_features, out_dim=1024)
teacher_head = DINOHead(in_dim=in_features, out_dim=1024)

student = DINO(encoder, student_head)
teacher = DINO(encoder, teacher_head)

for param in teacher.parameters():
    param.requires_grad = False

criterion = DINOLoss(out_dim=1024)

c:\Users\Adam\miniconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Adam\miniconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
def save_best_model(student, path):
    torch.save(student.state_dict(), path)
    
def save_losses(losses, path):
    with open(path, 'w') as f:
        for loss in losses:
            f.write(f'{loss}\n')
            
def load_model(student, path):
    student.load_state_dict(torch.load(path))
    return student

In [10]:
def plot_loss_curve(train_losses, val_losses):
    plt.figure(figsize=(12, 6))
    plt.plot(train_losses, label='train loss')
    plt.plot(val_losses, label='val loss')
    plt.title('DINO Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.legend()
    plt.show()

In [11]:
data_module = ReconstructionDataModule("../data", transforms.Compose([transforms.ToTensor()]))
data_module.setup()

In [12]:
def momentum_update(student, teacher, m=0.996):
    for param_q, param_k in zip(student.parameters(), teacher.parameters()):
        param_k.data = param_k.data * m + param_q.data * (1. - m)

def train_dino(student, teacher, criterion, optimizer, train_dataloader, val_dataloader, patient, num_epochs=20):
    student = student.to(device)
    teacher = teacher.to(device)
    criterion = criterion.to(device)
    train_losses = []
    val_losses = []
    patient_count = 0
    best_loss = float('inf')

    for epoch in range(num_epochs):
        epoch_train_loss = 0
        epoch_val_loss = 0
        for images, _ in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            student_outputs = [student(img.to(device)) for img in images]
            with torch.no_grad():
                teacher_outputs = [teacher(img.to(device)) for img in images]

            train_loss = 0
            n_crops = len(images)
            for i in range(n_crops):
                for j in range(n_crops):
                    if i != j:
                        loss = criterion(student_outputs[i], teacher_outputs[j])
                        train_loss += loss

            train_loss /= n_crops * (n_crops - 1)

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            momentum_update(student, teacher)
            epoch_train_loss += train_loss.item()
            
        with torch.no_grad():
            for images, _ in val_dataloader:
                images = [img.to(device) for img in images]
                student_outputs = [student(img) for img in images]
                teacher_outputs = [teacher(img) for img in images]

                val_loss = 0
                n_crops = len(images)
                for i in range(n_crops):
                    for j in range(n_crops):
                        if i != j:
                            loss = criterion(student_outputs[i], teacher_outputs[j])
                            val_loss += loss

                val_loss /= n_crops * (n_crops - 1)
                
                epoch_val_loss += val_loss.item()

        avg_train_loss = epoch_train_loss / len(train_dataloader)
        avg_val_loss = epoch_val_loss / len(val_dataloader)
        best_loss = min(best_loss, avg_train_loss)
        if avg_train_loss > best_loss:
            patient_count += 1
            if patient_count == patient:
                print(f'Early stopping at epoch {epoch+1}')
                return train_losses, val_losses

        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        if avg_train_loss == best_loss:
            save_best_model(student, '../ssl_methods/dino/best_dino.pth')
    return train_losses, val_losses


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
student = student.to(device)
teacher = teacher.to(device)
criterion = criterion.to(device)
optimizer = optim.Adam(student.parameters(), lr=1e-3)
num_epochs = 20
patience = 5
train_losses, val_losses = train_dino(student, teacher, criterion, optimizer, data_module.train_dataloader(), data_module.val_dataloader(), patience, num_epochs=num_epochs)

In [ ]:
plot_loss_curve(train_losses, val_losses)

In [14]:
class STLDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=128):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])
        self.path = "../data"

    def setup(self, stage=None):
        dataset = STL10(self.path, split="train", transform=self.transform, download=True)
        self.train_dataset, self.val_dataset = data_utils.random_split(dataset, [.8, .2])
        self.test_dataset = STL10(self.path, split="test", transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [15]:
class DinoLinearEval(pl.LightningModule):
    def __init__(self, learning_rate=1e-5):
        super().__init__()
        self.encoder = load_model(student, '../ssl_methods/dino/best_dino.pth').backbone
        self.model = nn.Sequential(self.encoder, nn.Linear(512, 10))
        self.loss_fn = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        self.accuracy = Accuracy(task="multiclass", num_classes=10)
        self.f1 = F1Score(num_classes=10, task="multiclass", average="macro")

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters())
        return optimizer

    def training_step(self, batch, batch_idx):
            img, label = batch
            output = self(img)
            loss = self.loss_fn(output, label)
            self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
            acc = self.accuracy(output, label)
            self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
            return loss

    def validation_step(self, batch, batch_idx):
        img, label = batch
        output = self(img)
        loss = self.loss_fn(output, label)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        acc = self.accuracy(output, label)
        self.log('val_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        img, label = batch
        output = self(img)
        loss = self.loss_fn(output, label)
        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        acc = self.accuracy(output, label)
        self.log('test_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        f1 = self.f1(output, label)
        self.log('test_f1', f1, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

In [17]:
batch_size = 64
learning_rate = 1e-3
torch.cuda.empty_cache()
stl10_dm = STLDataModule(batch_size=128)
stl10_dm.setup()

Files already downloaded and verified


In [ ]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='../ssl_methods/dino/checkpoints/',
    filename='dino-{batch_size}_{learning_rate}_{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='max',
)

trainer = pl.Trainer(
    max_epochs=20,
    callbacks=[checkpoint_callback]
)

train_module = DinoLinearEval(learning_rate)
trainer.fit(train_module, stl10_dm.train_dataloader(), stl10_dm.val_dataloader())

In [24]:
batch_sizes = [16, 32, 64, 128]
learning_rates = [5*1e-3, 1e-4, 0.5*1e-5]
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        data_module = STLDataModule(batch_size=batch_size)
        data_module.setup()
        model = DinoLinearEval(learning_rate=learning_rate)
        checkpoint_callback = ModelCheckpoint(
            monitor='val_loss',
            dirpath='../ssl_methods/dino/checkpoints/',
            filename='dino-linear-{epoch:02d}-{val_loss:.2f}',
            save_top_k=1,
            mode='min',
        )
        trainer = pl.Trainer(max_epochs=30, callbacks=[checkpoint_callback])
        trainer.fit(model, data_module)
        trainer.test(model, data_module.test_dataloader())

Files already downloaded and verified


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Missing logger folder: c:\Users\Adam\Desktop\PGM_UR\representation-learning-methods\notebooks\lightning_logs


Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | encoder  | ResNet             | 11.2 M | train
1 | model    | Sequential         | 11.2 M | train
2 | loss_fn  | CrossEntropyLoss   | 0      | train
3 | accuracy | MulticlassAccuracy | 0      | train
4 | f1       | MulticlassF1Score  | 0      | train
--------------------------------------------------------
5.1 K     Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=30` reached.


Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.31424999237060547
         test_f1            0.2952575385570526
        test_loss           1.8796669244766235
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
